<a href="https://colab.research.google.com/github/lawrencetang20/NLPproject/blob/main/NLP_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SECTION 1 - INSTALL/IMPORTS

**Install and import necessary packages; run all cells**

In [ ]:
!pip install pybaseball
!pip install transformers
!pip install optuna
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu116


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.1/426.1 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 375.9/375.9 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.4 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu116


In [ ]:
import os
import random
import warnings
from datetime import datetime, timedelta

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pybaseball import pitching_stats, statcast_pitcher, playerid_lookup

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, f1_score

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, GPT2ForSequenceClassification, Trainer, TrainingArguments

from tqdm import tqdm
import optuna


def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed()

START_YEAR = 2016
END_YEAR = 2024
TOP_N_PITCHERS = 8
EPOCHS = 5
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {DEVICE}")

Using device: cuda


In [ ]:
# inspecting data for a sample year

sample_year = 2016
try:
    sample_df = pitching_stats(sample_year)
    print(f"Columns in the data for year {sample_year}:")
    print(sample_df.columns.tolist())
    print("\nFirst 5 records:")
    display(sample_df.head())
except Exception as e:
    print(f"Error fetching data for year {sample_year}: {e}")


Columns in the data for year 2016:
['IDfg', 'Season', 'Name', 'Team', 'Age', 'W', 'L', 'WAR', 'ERA', 'G', 'GS', 'CG', 'ShO', 'SV', 'BS', 'IP', 'TBF', 'H', 'R', 'ER', 'HR', 'BB', 'IBB', 'HBP', 'WP', 'BK', 'SO', 'GB', 'FB', 'LD', 'IFFB', 'Balls', 'Strikes', 'Pitches', 'RS', 'IFH', 'BU', 'BUH', 'K/9', 'BB/9', 'K/BB', 'H/9', 'HR/9', 'AVG', 'WHIP', 'BABIP', 'LOB%', 'FIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH%', 'BUH%', 'Starting', 'Start-IP', 'Relieving', 'Relief-IP', 'RAR', 'Dollars', 'tERA', 'xFIP', 'WPA', '-WPA', '+WPA', 'RE24', 'REW', 'pLI', 'inLI', 'gmLI', 'exLI', 'Pulls', 'WPA/LI', 'Clutch', 'FB% 2', 'FBv', 'SL%', 'SLv', 'CT%', 'CTv', 'CB%', 'CBv', 'CH%', 'CHv', 'SF%', 'SFv', 'KN%', 'KNv', 'XX%', 'PO%', 'wFB', 'wSL', 'wCT', 'wCB', 'wCH', 'wSF', 'wKN', 'wFB/C', 'wSL/C', 'wCT/C', 'wCB/C', 'wCH/C', 'wSF/C', 'wKN/C', 'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'HLD', 'SD', 'MD', 'ERA-', 'FIP-', 'xFIP-', 'K%', 'B

,IDfg,Season,Name,Team,Age,W,L,WAR,ERA,G,...,Pit+ FC,Stf+ FS,Loc+ FS,Pit+ FS,Stuff+,Location+,Pitching+,Stf+ FO,Loc+ FO,Pit+ FO
6,11530,2016,Jose Fernandez,MIA,23,16,8,6.3,2.86,29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11762,2016,Noah Syndergaard,NYM,23,14,9,6.0,2.60,31,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,3137,2016,Max Scherzer,WSN,31,20,7,5.6,2.96,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,8700,2016,Justin Verlander,DET,33,16,9,5.4,3.04,34,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,10603,2016,Chris Sale,CHW,27,17,10,5.3,3.34,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# SECTION 2 - OBTAINING DATA

**Run all cells except saving data to google drive**

You can obtain the data yourself from pybaseball, but the pkl files are already provided

In [ ]:
# this function gets the top pitchers

def get_top_pitchers(start_year=START_YEAR, end_year=END_YEAR, top_n=TOP_N_PITCHERS):
    all_pitchers = []
    for year in range(start_year, end_year + 1):
        try:
            df = pitching_stats(year)
            all_pitchers.append(df)
        except Exception as e:
            print(f"Error fetching data for year {year}: {e}")

    if not all_pitchers:
        print("No data fetched. Please check the data source or parameters.")
        return []

    combined_df = pd.concat(all_pitchers, ignore_index=True)

    combined_df.columns = combined_df.columns.str.strip()

    print("\nColumns in the combined data:")
    print(combined_df.columns.tolist())

    required_columns = ['Name', 'IP']
    for col in required_columns:
        if col not in combined_df.columns:
            print(f"Required column '{col}' is missing in the data.")
            return []


    innings_pitched = combined_df.groupby('Name')['IP'].sum().reset_index()

    top_pitchers = innings_pitched.sort_values(by='IP', ascending=False).head(top_n)['Name'].tolist()

    print(f"\nTop {top_n} pitchers based on innings pitched:")
    for idx, pitcher in enumerate(top_pitchers, 1):
        print(f"{idx}. {pitcher}")

    return top_pitchers

top_pitchers = get_top_pitchers()



Columns in the combined data:
['IDfg', 'Season', 'Name', 'Team', 'Age', 'W', 'L', 'WAR', 'ERA', 'G', 'GS', 'CG', 'ShO', 'SV', 'BS', 'IP', 'TBF', 'H', 'R', 'ER', 'HR', 'BB', 'IBB', 'HBP', 'WP', 'BK', 'SO', 'GB', 'FB', 'LD', 'IFFB', 'Balls', 'Strikes', 'Pitches', 'RS', 'IFH', 'BU', 'BUH', 'K/9', 'BB/9', 'K/BB', 'H/9', 'HR/9', 'AVG', 'WHIP', 'BABIP', 'LOB%', 'FIP', 'GB/FB', 'LD%', 'GB%', 'FB%', 'IFFB%', 'HR/FB', 'IFH%', 'BUH%', 'Starting', 'Start-IP', 'Relieving', 'Relief-IP', 'RAR', 'Dollars', 'tERA', 'xFIP', 'WPA', '-WPA', '+WPA', 'RE24', 'REW', 'pLI', 'inLI', 'gmLI', 'exLI', 'Pulls', 'WPA/LI', 'Clutch', 'FB% 2', 'FBv', 'SL%', 'SLv', 'CT%', 'CTv', 'CB%', 'CBv', 'CH%', 'CHv', 'SF%', 'SFv', 'KN%', 'KNv', 'XX%', 'PO%', 'wFB', 'wSL', 'wCT', 'wCB', 'wCH', 'wSF', 'wKN', 'wFB/C', 'wSL/C', 'wCT/C', 'wCB/C', 'wCH/C', 'wSF/C', 'wKN/C', 'O-Swing%', 'Z-Swing%', 'Swing%', 'O-Contact%', 'Z-Contact%', 'Contact%', 'Zone%', 'F-Strike%', 'SwStr%', 'HLD', 'SD', 'MD', 'ERA-', 'FIP-', 'xFIP-', 'K%', 'BB%',

In [ ]:
# error with data of corbin and quintana (although code outputs show them), so only got the id 6 of the top 8 pitchers

START_GAME = '<START_GAME>'
START_INNING = '<START_INNING>'
START_ATBAT = '<START_ATBAT>'

name_to_id = {}

name_to_id['Aaron Nola'] = playerid_lookup('nola', 'aaron')['key_mlbam'][0]
name_to_id['Kevin Gausman'] = playerid_lookup('gausman', 'kevin')['key_mlbam'][0]
name_to_id['Gerrit Cole'] = playerid_lookup('cole', 'gerrit')['key_mlbam'][0]
name_to_id['Justin Verlander'] = playerid_lookup('verlander', 'justin')['key_mlbam'][0]
name_to_id['Jose Berrios'] = playerid_lookup('berríos', 'josé')['key_mlbam'][0]
name_to_id['Max Scherzer'] = playerid_lookup('scherzer', 'max')['key_mlbam'][0]

print(name_to_id)

Gathering player lookup table. This may take a moment.
{'Aaron Nola': 605400, 'Kevin Gausman': 592332, 'Gerrit Cole': 543037, 'Justin Verlander': 434378, 'Jose Berrios': 621244, 'Max Scherzer': 453286}


In [ ]:
warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

# gets pitch data

def get_all_pitch_data(player_name, start_year, end_year, name_to_id):
    mlbam_id = name_to_id[player_name]
    data = statcast_pitcher(start_dt=f'{start_year}-01-01', end_dt=f'{end_year}-10-05', player_id=mlbam_id)

    columns_to_keep = [
            'pitch_type', 'balls', 'strikes', 'outs_when_up', 'fld_score', 'bat_score',
            'stand', 'p_throws', 'inning', 'description', 'on_1b', 'on_2b', 'on_3b', 'game_year', 'game_date'
        ]
    data = data[columns_to_keep].copy()

    pitch_mapping = {
        'FF': 'Fastball', 'SI': 'Fastball', 'IN': 'Fastball', 'EP': 'Fastball',
        'FC': 'Fastball', 'PO': 'Fastball',  # Fastballs
        'SL': 'Breaking Ball', 'CU': 'Breaking Ball', 'KC': 'Breaking Ball', 'SV': 'Breaking Ball', 'ST': 'Breaking Ball', # Breaking Balls
        'CH': 'Off Speed', 'FS': 'Off Speed', 'FO': 'Off Speed'  # Offspeeds
    }

    data['pitch_type'] = data['pitch_type'].map(pitch_mapping).fillna('Fastball')

    counter = 0
    counters = [None] * len(data)

    for i in range(len(data) - 1, -1, -1):
        row = data.iloc[i]
        if row['balls'] == 0 and row['strikes'] == 0:
            counter += 1
        counters[i] = counter

    data['at_bat'] = counters
    return data


# SECTION 2.1 - OPTIONAL*

**YOU DO NOT NEED TO RUN THIS CELL, ONLY IF YOU WANT TO SAVE THE DATA YOURSELF, BUT IT IS PROVIDED IN THE PKL FILES**

This process was already done by our team to save time in fetching data from pybaseball

In [ ]:
# run this cell to save the data to google drive

from google.colab import drive
drive.mount('/content/drive')

for pitcher in name_to_id:
  print(f'{pitcher}')
  player_data = get_all_pitch_data(pitcher, 2016, 2024, name_to_id=name_to_id)
  name = pitcher.replace(" ", "_")
  print(f'/content/drive/MyDrive/{name}.pkl')

  player_data.to_pickle(f'/content/drive/MyDrive/{name}.pkl')


KeyboardInterrupt: 

# SECTION 3 - USING DATA FROM PKL

**How to load data from google drive; run this whole section**

1) put the downloaded pkl files from the zip into your google drive

2) connect drive to collab

3) run this cell to get the data into collab

In [ ]:
# get the data from google drive

from google.colab import drive
drive.mount('/content/drive')

def load_pitcher_data(pitcher_name):
  name = pitcher_name.replace(" ", "_")
  df = pd.read_pickle(f'/content/drive/MyDrive/{name}.pkl')
  return df

# test if it loaded correctly
dat = load_pitcher_data('Aaron Nola')
print(dat.head(5))

Mounted at /content/drive
      pitch_type  balls  strikes  outs_when_up  fld_score  bat_score stand  \
0       Fastball      0        2             2          2          3     L   
1       Fastball      0        1             2          2          3     L   
2  Breaking Ball      0        0             2          2          3     L   
3       Fastball      0        1             1          2          3     R   
4       Fastball      0        0             1          2          3     R   

  p_throws  inning      description  on_1b  on_2b  on_3b  game_year  \
0        R       6  swinging_strike    NaN    NaN    NaN       2021   
1        R       6  swinging_strike    NaN    NaN    NaN       2021   
2        R       6    called_strike    NaN    NaN    NaN       2021   
3        R       6    hit_into_play    NaN    NaN    NaN       2021   
4        R       6             foul    NaN    NaN    NaN       2021   

    game_date  at_bat  
0  2021-09-29    7125  
1  2021-09-29    7125  
2  202

# SECTION 4 - DEFINING FUNCTIONS FOR MODELS

**Run all cells**

In [ ]:
# all different sentence functions

def convert_pitch_to_long_sentence(pitch):
  """
  pitch: row in PD dataframe
  """
  runners_on = []
  if pitch['on_1b'] == 1:
      runners_on.append("first")
  if pitch['on_2b'] == 1:
      runners_on.append("second")
  if pitch['on_3b'] == 1:
      runners_on.append("third")

  if runners_on:
        bases = " and ".join(runners_on) + " base(s)"
  else:
      bases = "no base"

  pitch_type = pitch['pitch_type']
  balls = pitch['balls']
  strikes = pitch['strikes']
  outs = pitch['outs_when_up']
  score = pitch['fld_score']
  bat_score = pitch['bat_score']
  fld_score = pitch['fld_score']
  stand = pitch['stand']
  p_throws = pitch['p_throws']
  inning = pitch['inning']
  description = pitch['description']
  game_year = pitch['game_year']

  sentence = (f"In inning {inning}, with {balls} balls, {strikes} strikes, "
        f"and {outs} outs, the batter, in a {stand}-handed stance, "
        f"faced a {p_throws}-handed pitcher. The field score was {fld_score}, "
        f"and the batter score was {bat_score}. There were runners on {bases}, "
        f"and the pitch type was {pitch_type}, resulting in {description}."
  )

  return sentence

def convert_pitch_to_short_sentence(pitch):
  pitch_type = pitch['pitch_type']
  balls = pitch['balls']
  strikes = pitch['strikes']
  description = pitch['description']

  sentence = (f"With {balls} balls and {strikes} strikes, "
        f"Pitcher throws a pitch with pitch type {pitch_type}, resulting in {description}."
  )

  return sentence

def just_ball_strike(pitch):
  pitch_type = pitch['pitch_type']
  balls = pitch['balls']
  strikes = pitch['strikes']
  description = pitch['description']

  sentence = (f"{balls} - {strikes}: {pitch_type}"
  )

  return sentence

def just_pitch_type(pitch):
  pitch_type = pitch['pitch_type']
  return pitch_type

def features_only_no_other_words(pitch):
  """
  pitch: row in PD dataframe
  """
  runners_on = []
  if pitch['on_1b'] == 1:
      runners_on.append("first")
  if pitch['on_2b'] == 1:
      runners_on.append("second")
  if pitch['on_3b'] == 1:
      runners_on.append("third")

  if runners_on:
        bases = " and ".join(runners_on) + " base(s)"
  else:
      bases = "no base"

  pitch_type = pitch['pitch_type']
  balls = pitch['balls']
  strikes = pitch['strikes']
  outs = pitch['outs_when_up']
  score = pitch['fld_score']
  bat_score = pitch['bat_score']
  fld_score = pitch['fld_score']
  stand = pitch['stand']
  p_throws = pitch['p_throws']
  inning = pitch['inning']
  description = pitch['description']
  game_year = pitch['game_year']

  sentence = (f"{inning} {balls} {strikes} {outs} {stand} {p_throws} {fld_score} {bat_score} {bases} {pitch_type} {description}"
  )

  return sentence

def get_pitch_sentence_df(player_data, sentence_func):
  player_data['nlp_pitch_description'] = player_data.apply(sentence_func, axis=1)

  new_df = player_data[['pitch_type', 'at_bat', 'nlp_pitch_description']].copy()

  return new_df[::-1].reset_index(drop=True)

In [ ]:
# function to train model

def train_model(player, context_size, sentence_func, train_size, test_size):
  player_data = load_pitcher_data(player)

  data = get_pitch_sentence_df(player_data, sentence_func)

  def create_context_data_sliding(final_df, window_size):
      samples = []
      for i in range(0, len(final_df) - window_size):
          context = " ".join(final_df.iloc[i + j]['nlp_pitch_description'] for j in range(window_size))
          target_pitch = final_df.iloc[i + window_size]['pitch_type']
          samples.append((context, target_pitch))
      return samples

  context_samples = create_context_data_sliding(data, context_size)

  train_samples, test_samples = train_test_split(context_samples, train_size = train_size, test_size=test_size, random_state=42)

  pitch_types = data['pitch_type'].unique().tolist()
  label2id = {pitch: idx for idx, pitch in enumerate(pitch_types)}
  id2label = {idx: pitch for pitch, idx in label2id.items()}

  train_contexts, train_targets = zip(*train_samples)
  test_contexts, test_targets = zip(*test_samples)

  train_labels = [label2id[target] for target in train_targets]
  test_labels = [label2id[target] for target in test_targets]

  tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
  tokenizer.add_special_tokens({'pad_token': '[PAD]'})  # Add a pad token

  model = GPT2ForSequenceClassification.from_pretrained("gpt2", num_labels=len(pitch_types))
  model.resize_token_embeddings(len(tokenizer))  # Resize embeddings to match tokenizer

  model.config.pad_token_id = tokenizer.pad_token_id  # Add this line

  def tokenize_function(contexts):
      return tokenizer(
          list(contexts),
          padding=True,
          truncation=True,
          max_length=512
      )

  train_encodings = tokenize_function(train_contexts)
  test_encodings = tokenize_function(test_contexts)

  class PitchClassificationDataset(torch.utils.data.Dataset):
      def __init__(self, encodings, labels):
          self.encodings = encodings
          self.labels = labels

      def __getitem__(self, idx):
          item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
          item['labels'] = torch.tensor(self.labels[idx])
          return item

      def __len__(self):
          return len(self.labels)

  train_dataset = PitchClassificationDataset(train_encodings, train_labels)
  test_dataset = PitchClassificationDataset(test_encodings, test_labels)

  def compute_metrics_classification(eval_pred):
      logits, labels = eval_pred
      predictions = np.argmax(logits, axis=1)
      acc = accuracy_score(labels, predictions)
      f1 = f1_score(labels, predictions, average='weighted')
      return {'accuracy': acc, 'f1': f1}

  training_args = TrainingArguments(
      output_dir="./results",
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      num_train_epochs=3,
      logging_dir="./logs",
      logging_steps=50,
      eval_strategy="steps",
      eval_steps=500,
      save_steps=500,
      save_total_limit=2,
      load_best_model_at_end=True,
      metric_for_best_model="accuracy",
      greater_is_better=True,
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train_dataset,
      eval_dataset=test_dataset,
      compute_metrics=compute_metrics_classification,
  )

  trainer.train()

  eval_results = trainer.evaluate()
  print(f"{player} Evaluation results with {sentence_func} and context_size {context_size}:", eval_results)
  return eval_results['eval_accuracy']


# SECTION 5 - RUNNING MODELS

**How to run the models: REQUIRES USER TO COPY AND PASTE**

1) run the initilization cell below which stores all the accuracies

2) run the unigram models to obtain a baseline

3) follow the commented out format and copy and paste to run the models

4) for *CONTEXT_SIZE*, choose between [2, 4, 6, 8, 10, 12, 16] because 0 is unigram model

5) for *TYPE_OF_SENTENCE*, choose between [convert_pitch_to_long_sentence, convert_pitch_to_short_sentence, just_ball_strike, just_pitch_type, features_only_no_other_words]

NOTE: only one example model was shown because model training time takes 20-30 mins, and 70+ models were trained

NOTE: you also need to make a wandb.ai account to use the api, it is free and you follow the directions when running the model with the above format

NOTE: all the data and statistics are saved in the google sheet

EX: there is an example below with *CONTEXT_SIZE* of 8 with *TYPE_OF SENTENCE* of convert_pitch_to_short_sentence

In [ ]:
# initialize the accuracies dictionary for trained models

pitcher_accuracies = {}

for pitcher in name_to_id.keys():
  print(pitcher)
  pitcher_accuracies[pitcher] = {-1: "initialized"}

Aaron Nola
Kevin Gausman
Gerrit Cole
Justin Verlander
Jose Berrios
Max Scherzer


In [ ]:
# unigram model for baseline comparison

warnings.filterwarnings("ignore", category=pd.errors.DtypeWarning)

def unigram_test(name_to_id):
  for pitcher in name_to_id.keys():
    all_data = load_pitcher_data(pitcher)

    train_data, test_data = train_test_split(all_data, train_size = 2500, test_size=300, random_state=42)

    most_common_pitch = train_data['pitch_type'].mode().iloc[0]
    print(f"Most common pitch for {pitcher}: {most_common_pitch}")

    accuracy = (test_data['pitch_type'] == most_common_pitch).mean()
    print(f"Accuracy of predicting {most_common_pitch} for {pitcher}: {accuracy:.2%}")
    print("\n")

    pitcher_accuracies[pitcher][0] = accuracy

unigram_test(name_to_id)


Most common pitch for Aaron Nola: Fastball
Accuracy of predicting Fastball for Aaron Nola: 47.33%


Most common pitch for Kevin Gausman: Fastball
Accuracy of predicting Fastball for Kevin Gausman: 56.00%


Most common pitch for Gerrit Cole: Fastball
Accuracy of predicting Fastball for Gerrit Cole: 57.67%


Most common pitch for Justin Verlander: Fastball
Accuracy of predicting Fastball for Justin Verlander: 56.33%


Most common pitch for Jose Berrios: Fastball
Accuracy of predicting Fastball for Jose Berrios: 54.00%


Most common pitch for Max Scherzer: Fastball
Accuracy of predicting Fastball for Max Scherzer: 54.67%




In [ ]:
"""

format for running models

for pitcher in name_to_id:
  print(pitcher)
  accuracy = train_model(pitcher, CONTEXT_SIZE, TYPE_OF_SENTENCE, train_size=2500, test_size=300)
  pitcher_accuracies[pitcher][CONTEXT_SIZE] = accuracy

"""

In [ ]:
# example running of model
# this model was run to obtain all the data in the attached google sheet numerous times

for pitcher in name_to_id:
  print(pitcher)
  accuracy = train_model(pitcher, 8, convert_pitch_to_short_sentence, train_size=2500, test_size=300)
  # note that it does not store different types of sentences, so we advise you run the model with the same type of sentence for all pitchers first, then re initialize and restart
  pitcher_accuracies[pitcher][8] = accuracy

Aaron Nola


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


Aaron Nola Evaluation results with <function convert_pitch_to_short_sentence at 0x7a870c38e170> and context_size 8: {'eval_loss': 0.9581496119499207, 'eval_accuracy': 0.54, 'eval_f1': 0.38452631578947366, 'eval_runtime': 3.5491, 'eval_samples_per_second': 84.528, 'eval_steps_per_second': 5.353, 'epoch': 3.0}
Kevin Gausman


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

Step,Training Loss,Validation Loss


Jose Quintana Evaluation results with <function convert_pitch_to_short_sentence at 0x7a870c38e170> and context_size 8: {'eval_loss': 0.9161618947982788, 'eval_accuracy': 0.5866666666666667, 'eval_f1': 0.43383753501400557, 'eval_runtime': 3.7075, 'eval_samples_per_second': 80.918, 'eval_steps_per_second': 5.125, 'epoch': 3.0}
Max Scherzer


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


Max Scherzer Evaluation results with <function convert_pitch_to_short_sentence at 0x7a870c38e170> and context_size 8: {'eval_loss': 0.9265761375427246, 'eval_accuracy': 0.5833333333333334, 'eval_f1': 0.4298245614035088, 'eval_runtime': 3.5775, 'eval_samples_per_second': 83.858, 'eval_steps_per_second': 5.311, 'epoch': 3.0}


In [ ]:
print(pitcher_accuracies)

{'Aaron Nola': {-1: 'initialized', 0: 0.47333333333333333}, 'Kevin Gausman': {-1: 'initialized', 0: 0.56}, 'Gerrit Cole': {-1: 'initialized', 0: 0.5766666666666667}, 'Justin Verlander': {-1: 'initialized', 0: 0.5633333333333334}, 'Jose Berrios': {-1: 'initialized', 0: 0.54}, 'Max Scherzer': {-1: 'initialized', 0: 0.5466666666666666}}
